# Initial testing of the model.
Referred [Time-Series Prediction by "Hvass Laboratories"](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/23_Time-Series-Prediction.ipynb)

## Basic Imports

In [16]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

## Data Preparation

In [20]:
df = pd.read_csv('../Data/cleanned/zusammen.csv')
df['date'] = pd.to_datetime(df['date'])
df['days'] = df['date'].dt.dayofyear
df = df.drop(['rohwert'], axis=1)

## Data sorting and splitting

In [23]:
yDataSet = df.pop('wert')
xDataSet = df

In [25]:
xTrain, xTest, yTrain, yTest = train_test_split(xDataSet,
                                yDataSet, 
                                test_size=0.3, 
                                random_state=42)

In [26]:
xTrain.shape, xTest.shape, yTrain.shape, yTest.shape

((16737, 6), (7173, 6), (16737,), (7173,))